In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [6]:
data_path = "../data/uci_data.xls"

In [7]:
df = pd.read_excel(data_path, header=0, index_col=0, skiprows=1)

In [19]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train.drop(columns=("default payment next month"))
y_train = train[["default payment next month"]]
X_test = test.drop(columns=("default payment next month"))
y_test = test[["default payment next month"]]

In [20]:
train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
21754,80000,2,2,2,24,0,0,0,0,0,...,73731,39643,39457,3503,5001,2092,1218,1445,878,0
252,30000,1,2,2,28,0,0,0,0,0,...,25255,22001,0,5006,1244,851,955,0,0,0
22942,180000,2,5,1,44,0,0,-1,-1,-1,...,0,6881,10340,0,850,0,6881,10340,182,0
619,60000,1,1,2,25,0,0,0,0,0,...,39639,39619,39140,2018,1900,2000,1500,1900,2000,0
17091,130000,2,2,2,25,0,0,0,0,0,...,117823,120854,123904,4100,4200,5000,5000,5000,10700,0


In [12]:
cat = ["SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
num = ["LIMIT_BAL", "AGE", 
       "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", 
       "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]

scaler = preprocessing.StandardScaler().fit(train[num])

In [13]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
x_train_scaled = scaler.transform(train[num])

In [15]:
x_train_scaled

array([[-0.67235902, -1.2446578 ,  0.32966974, ..., -0.22465419,
        -0.21600791, -0.24524018],
       [-1.05776818, -0.81152149, -0.29544512, ..., -0.24078483,
        -0.30852284, -0.29553162],
       [ 0.0984593 ,  0.92102376, -0.40887943, ...,  0.12267581,
         0.35348711, -0.28510675],
       ...,
       [-0.90360452, -1.02808964, -0.69384123, ..., -0.29935805,
        -0.30852284, -0.29553162],
       [-0.74944086, -1.13637372,  0.31351155, ..., -0.22575819,
        -0.30852284, -0.22679617],
       [-0.05570437,  0.05475114, -0.69411371, ..., -0.19815824,
         0.60062042, -0.20961231]])

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [39]:
numeric_features = ["LIMIT_BAL", "AGE", 
                    "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", 
                    "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ["SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs', max_iter=500))])

clf.fit(X_train, y_train.values.ravel())

print("model score: %.3f" % clf.score(X_test, y_test.values))

model score: 0.819


In [53]:
w = categorical_transformer.fit_transform(X_train[categorical_features])

In [55]:
w.todense()

matrix([[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]])

In [47]:
w.toarray()

MemoryError: Unable to allocate 26.1 GiB for an array with shape (24000, 145920) and data type float64